In [2]:
from flask import Flask, request, render_template_string
from threading import Thread
from werkzeug.serving import make_server

# Flask app initialization
app = Flask(__name__)

# HTML template with CSS for styling
html_template = '''
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Customer Churn Prediction</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            background-color: #f4f4f4;
            margin: 0;
            padding: 20px;
        }
        h2 {
            text-align: center;
            color: #333;
        }
        form {
            background-color: #fff;
            max-width: 600px;
            margin: 20px auto;
            padding: 20px;
            border-radius: 8px;
            box-shadow: 0px 0px 10px rgba(0, 0, 0, 0.1);
        }
        label {
            font-weight: bold;
            display: block;
            margin-top: 10px;
        }
        input[type="text"], input[type="number"] {
            width: 100%;
            padding: 10px;
            margin-top: 5px;
            margin-bottom: 15px;
            border: 1px solid #ccc;
            border-radius: 4px;
        }
        input[type="submit"] {
            width: 100%;
            padding: 12px;
            background-color: #28a745;
            color: white;
            border: none;
            border-radius: 4px;
            cursor: pointer;
            font-size: 16px;
        }
        input[type="submit"]:hover {
            background-color: #218838;
        }
        .result {
            margin-top: 20px;
            text-align: center;
            font-size: 20px;
            color: #333;
        }
    </style>
</head>
<body>
    <h2>Customer Churn Prediction</h2>
    <form action="/predict" method="POST">
        <label for="gender">Gender (Male/Female):</label>
        <input type="text" id="gender" name="gender" required>

        <label for="SeniorCitizen">Senior Citizen (Yes/No):</label>
        <input type="text" id="SeniorCitizen" name="SeniorCitizen" required>

        <label for="Partner">Partner (Yes/No):</label>
        <input type="text" id="Partner" name="Partner" required>

        <label for="Dependents">Dependents (Yes/No):</label>
        <input type="text" id="Dependents" name="Dependents" required>

        <label for="tenure">Tenure (months):</label>
        <input type="number" id="tenure" name="tenure" required>

        <label for="PhoneService">Phone Service (Yes/No):</label>
        <input type="text" id="PhoneService" name="PhoneService" required>

        <label for="MultipleLines">Multiple Lines (Yes/No):</label>
        <input type="text" id="MultipleLines" name="MultipleLines" required>

        <label for="InternetService">Internet Service (DSL/Fiber optic/No):</label>
        <input type="text" id="InternetService" name="InternetService" required>

        <label for="OnlineSecurity">Online Security (Yes/No):</label>
        <input type="text" id="OnlineSecurity" name="OnlineSecurity" required>

        <label for="OnlineBackup">Online Backup (Yes/No):</label>
        <input type="text" id="OnlineBackup" name="OnlineBackup" required>

        <label for="DeviceProtection">Device Protection (Yes/No):</label>
        <input type="text" id="DeviceProtection" name="DeviceProtection" required>

        <label for="TechSupport">Tech Support (Yes/No):</label>
        <input type="text" id="TechSupport" name="TechSupport" required>

        <label for="StreamingTV">Streaming TV (Yes/No):</label>
        <input type="text" id="StreamingTV" name="StreamingTV" required>

        <label for="StreamingMovies">Streaming Movies (Yes/No):</label>
        <input type="text" id="StreamingMovies" name="StreamingMovies" required>

        <label for="Contract">Contract (Month-to-month/One year/Two year):</label>
        <input type="text" id="Contract" name="Contract" required>

        <label for="PaperlessBilling">Paperless Billing (Yes/No):</label>
        <input type="text" id="PaperlessBilling" name="PaperlessBilling" required>

        <label for="PaymentMethod">Payment Method (Electronic check/Mailed check/Bank transfer/Credit card):</label>
        <input type="text" id="PaymentMethod" name="PaymentMethod" required>

        <label for="MonthlyCharges">Monthly Charges:</label>
        <input type="number" id="MonthlyCharges" name="MonthlyCharges" step="any" required>

        <label for="TotalCharges">Total Charges:</label>
        <input type="number" id="TotalCharges" name="TotalCharges" step="any" required>

        <input type="submit" value="Predict">
    </form>

    {% if prediction_text %}
    <div class="result">
        <h3>{{ prediction_text }}</h3>
    </div>
    {% endif %}
</body>
</html>
'''

# Function to count "Yes" and "No" in the form
def count_yes_no(data):
    yes_count = 0
    no_count = 0

    # Checking fields where Yes/No responses are expected
    for key, value in data.items():
        if value.lower() == 'yes':
            yes_count += 1
        elif value.lower() == 'no':
            no_count += 1

    return yes_count, no_count

# Define index route to display the form
@app.route('/')
def index():
    return render_template_string(html_template)

# Define prediction route
@app.route('/predict', methods=['POST'])
def predict():
    try:
        # Get data from the form
        data = request.form
        
        # Count "Yes" and "No"
        yes_count, no_count = count_yes_no(data)

        # Compare counts and make the prediction
        if yes_count > no_count:
            result = "This customer will churn"
        else:
            result = "This customer will not churn"

        return render_template_string(html_template, prediction_text=f'{result}')
    
    except Exception as e:
        print(f"Error: {e}")  # Log any error
        return render_template_string(html_template, prediction_text='Error occurred during prediction.')

# Start Flask in a separate thread
class FlaskThread(Thread):
    def __init__(self, app):
        Thread.__init__(self)
        self.srv = make_server('127.0.0.1', 5000, app)
        self.ctx = app.app_context()
        self.ctx.push()

    def run(self):
        self.srv.serve_forever()

    def shutdown(self):
        self.srv.shutdown()

if __name__ == '__main__':
    FlaskThread(app).start()

#http://127.0.0.1:5000/
